# Group Project 3

### Group - 2

1) Neil Vashani

2) Kousik Nandury

3) Venkata Sai Teja

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

pyo.init_notebook_mode(connected=True)

## Loading and Inspection

In [ ]:
file_path = '1.08_Crash_Data_Report_(detail).csv'
df = pd.read_csv(file_path)


In [ ]:
print(f"Dataset Shape: {df.shape}")
print(f"Total Records: {df.shape[0]:,}")
print(f"Total Columns: {df.shape[1]}")

Dataset Shape: (51305, 35)
Total Records: 51,305
Total Columns: 35


In [ ]:
print("\nColumn Information:")
print(df.info())



Column Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51305 entries, 0 to 51304
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   X                    51162 non-null  float64
 1   Y                    51162 non-null  float64
 2   OBJECTID             51305 non-null  int64  
 3   Incidentid           51304 non-null  float64
 4   DateTime             51304 non-null  object 
 5   Year                 51304 non-null  float64
 6   StreetName           51282 non-null  object 
 7   CrossStreet          50817 non-null  object 
 8   Distance             51303 non-null  float64
 9   JunctionRelation     51303 non-null  object 
 10  Totalinjuries        51303 non-null  float64
 11  Totalfatalities      51303 non-null  float64
 12  Injuryseverity       51303 non-null  object 
 13  Collisionmanner      51303 non-null  object 
 14  Lightcondition       51303 non-null  object 
 15  Weather        

In [ ]:
display(df.head())

,X,Y,OBJECTID,Incidentid,DateTime,Year,StreetName,CrossStreet,Distance,JunctionRelation,...,Unittype_Two,Age_Drv2,Gender_Drv2,Traveldirection_Two,Unitaction_Two,Violation1_Drv2,AlcoholUse_Drv2,DrugUse_Drv2,Latitude,Longitude
0,-111.95221,33.42191,1,2598369.0,2012/01/01 07:25:00+00,2012.0,University Dr,Hardy Dr,0.000,Intersection Non Interchange,...,Driver,20.0,Female,East,Going Straight Ahead,No Improper Action,No Apparent Influence,No Apparent Influence,33.42191,-111.95221
1,-111.93769,33.42319,2,2593420.0,2012/01/01 11:45:00+00,2012.0,7th St,Myrtle Ave,139.920,Not Junction Related,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.42319,-111.93769
2,-111.89196,33.41842,3,2588493.0,2012/01/01 18:07:00+00,2012.0,Price Rd,Orange St,0.000,Intersection Interchange,...,Driver,31.0,Male,South,Going Straight Ahead,No Improper Action,No Apparent Influence,No Apparent Influence,33.41842,-111.89196
3,-111.94791,33.41779,4,2584449.0,2012/01/02 00:03:00+00,2012.0,11th St,Judd St,200.112,Not Junction Related,...,Driverless,NaN,NaN,East,Properly Parked,NaN,NaN,NaN,33.41779,-111.94791
4,-111.92847,33.34956,5,2592211.0,2012/01/02 00:12:00+00,2012.0,Rural Rd,Elliot Rd,135.168,Intersection Related Non Interchange,...,Driver,20.0,Female,South,Going Straight Ahead,No Improper Action,No Apparent Influence,No Apparent Influence,33.34956,-111.92847


In [ ]:
print("\nGeospatial Columns Available:")
geo_cols = ['X', 'Y', 'Latitude', 'Longitude']
for col in geo_cols:
    if col in df.columns:
        print(f" {col}: {df[col].count()} non-null values")


Geospatial Columns Available:
 X: 51162 non-null values
 Y: 51162 non-null values
 Latitude: 51162 non-null values
 Longitude: 51162 non-null values


In [ ]:

numeric_cols = ['Totalinjuries', 'Totalfatalities', 'Age_Drv1', 'Age_Drv2']
for col in numeric_cols:
    if col in df.columns:
        print(f"{col}: Mean={df[col].mean():.2f}, Max={df[col].max()}, Min={df[col].min()}")

Totalinjuries: Mean=0.46, Max=9.0, Min=0.0
Totalfatalities: Mean=0.00, Max=3.0, Min=0.0
Age_Drv1: Mean=47.09, Max=255.0, Min=2.0
Age_Drv2: Mean=39.27, Max=255.0, Min=2.0


## Missing values:

In [ ]:
# Check missing values
print("Missing Values Analysis:")
missing_summary = df.isnull().sum()
missing_percent = (missing_summary / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing_summary,
    'Missing_Percentage': missing_percent
}).sort_values('Missing_Percentage', ascending=False)

print(missing_df[missing_df['Missing_Count'] > 0].head(10))

Missing Values Analysis:
                     Missing_Count  Missing_Percentage
Gender_Drv2                   4858            9.468863
DrugUse_Drv2                  4758            9.273950
Age_Drv2                      4758            9.273950
AlcoholUse_Drv2               4758            9.273950
Violation1_Drv2               4758            9.273950
Unittype_Two                  3389            6.605594
Unitaction_Two                3389            6.605594
Traveldirection_Two           3389            6.605594
Gender_Drv1                    930            1.812689
CrossStreet                    488            0.951174


## Data Cleaning:


In [ ]:

df_clean = df.copy()

initial_count = len(df_clean)
df_clean = df_clean.dropna(subset=['Latitude', 'Longitude'])
print(f"Removed {initial_count - len(df_clean)} rows with missing coordinates")

df_clean = df_clean[
    (df_clean['Latitude'].between(25, 50)) &
    (df_clean['Longitude'].between(-130, -65))
]
print(f"Final dataset shape after coordinate cleaning: {df_clean.shape}")


Removed 143 rows with missing coordinates
Final dataset shape after coordinate cleaning: (51162, 35)


In [ ]:
# Removing  incorrect coordinates
df_clean = df_clean[
    (df_clean['Latitude'].between(25, 50)) &
    (df_clean['Longitude'].between(-130, -65))
]
print(f"Final dataset shape after coordinate cleaning: {df_clean.shape}")


if 'DateTime' in df_clean.columns:
    df_clean['DateTime'] = pd.to_datetime(df_clean['DateTime'], errors='coerce')
    df_clean['Hour'] = df_clean['DateTime'].dt.hour
    df_clean['DayOfWeek'] = df_clean['DateTime'].dt.day_name()
    df_clean['Month'] = df_clean['DateTime'].dt.month


Final dataset shape after coordinate cleaning: (51162, 35)


In [ ]:
# Clean categorical columns
categorical_cols = ['Weather', 'Lightcondition', 'SurfaceCondition',
                   'Collisionmanner', 'Injuryseverity']
for col in categorical_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna('Unknown')

# Clean numeric columns
numeric_cols = ['Totalinjuries', 'Totalfatalities', 'Age_Drv1', 'Age_Drv2']
for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(0)



## Data Viz:

### Density map of total injuries:

In [ ]:
# Sampleing data as plot becomes crowded
if len(df_clean) > 10000:
    df_sample = df_clean.sample(n=10000, random_state=42)
    print(f"Sampling {len(df_sample)} points for visualization performance")
else:
    df_sample = df_clean

density_map = px.density_mapbox(
    df_sample,
    lat='Latitude',
    lon='Longitude',
    z='Totalinjuries',
    radius=10,
    center=dict(lat=df_clean['Latitude'].mean(), lon=df_clean['Longitude'].mean()),
    zoom=10,
    mapbox_style="open-street-map",
    title="Crash Density Map - Total Injuries",
    color_continuous_scale="YlOrRd"
)

density_map.update_layout(
    height=600,
    title_x=0.5,
    title_font_size=16
)




density_map.write_html("density_map_visualization.html")
print("Saved density_map_visualization.html")

Sampling 10000 points for visualization performance
Saved density_map_visualization.html


Insights:

- Majority of the junctions show hight total injuries with bright red spots on the map.

## Desity map of severity:

In [ ]:
# Sample data for performance
if len(df_clean) > 5000:
    df_sample_severity = df_clean.sample(n=5000, random_state=42)
else:
    df_sample_severity = df_clean

# Create severity categories
df_sample_severity['Severity_Category'] = pd.cut(
    df_sample_severity['Totalinjuries'] + df_sample_severity['Totalfatalities'],
    bins=[-1, 0, 2, 5, float('inf')],
    labels=['No Injury', 'Minor', 'Moderate', 'Severe']
)

severity_map = px.scatter_mapbox(
    df_sample_severity,
    lat='Latitude',
    lon='Longitude',
    color='Severity_Category',
    size='Totalinjuries',
    size_max=15,
    hover_data=['DateTime', 'Weather', 'Lightcondition'],
    mapbox_style="open-street-map",
    title="Crash Locations by Severity",
    center=dict(lat=df_clean['Latitude'].mean(), lon=df_clean['Longitude'].mean()),
    zoom=10
)

severity_map.update_layout(
    height=600,
    title_x=0.5,
    title_font_size=16
)



severity_map.write_html("severity_map_visualization.html")
print("Saved: severity_map_visualization.html")

Saved: severity_map_visualization.html


Insights:


- Crashes labeled as “Severe” are more geographically concentrated than those labeled as “Moderate” or “Minor”, suggesting potential environmental or behavioral factors unique to those areas.

- Larger markers correspond to higher total injuries, helping identify not just frequent but also high-impact locations.

- This helps differentiate between locations with frequent minor crashes vs. fewer but more harmful incidents

## Temporal analysis Subplots: bar plots

In [ ]:
temporal_fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Crashes by Hour of Day', 'Crashes by Day of Week',
                   'Crashes by Month', 'Injury Severity by Hour'],
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# Hour analysis
hourly_crashes = df_clean.groupby('Hour').size()
temporal_fig.add_trace(
    go.Bar(x=hourly_crashes.index, y=hourly_crashes.values, name='Hourly Crashes'),
    row=1, col=1
)

# Day of week analysis
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_crashes = df_clean['DayOfWeek'].value_counts().reindex(day_order)
temporal_fig.add_trace(
    go.Bar(x=daily_crashes.index, y=daily_crashes.values, name='Daily Crashes'),
    row=1, col=2
)

# Monthly analysis
monthly_crashes = df_clean.groupby('Month').size()
temporal_fig.add_trace(
    go.Bar(x=monthly_crashes.index, y=monthly_crashes.values, name='Monthly Crashes'),
    row=2, col=1
)

# Severity by hour
severity_hourly = df_clean.groupby('Hour')['Totalinjuries'].mean()
temporal_fig.add_trace(
    go.Bar(x=severity_hourly.index, y=severity_hourly.values, name='Avg Injuries by Hour'),
    row=2, col=2
)

temporal_fig.update_layout(
    height=800,
    title_text="Temporal Analysis of Crash Data",
    title_x=0.5,
    showlegend=False
)



temporal_fig.write_html("temporal_analysis_visualization.html")
print("Saved temporal_analysis_visualization.html")

Saved temporal_analysis_visualization.html


Insights:

- Majority of the crashes happen during night times during the times of 10pm to 2 am. And are the least at 10am.

- When analysed by the week , thursdays and fridays have registered the highest crashes.

- And october has registered the highest crashes. Meaning that nights , weekends and holiday times are the peak times for crashes.

## Weather analysis : Bar plot

In [ ]:

weather_stats = df_clean.groupby('Weather').agg({
    'Incidentid': 'count',
    'Totalinjuries': 'mean',
    'Totalfatalities': 'mean'
}).round(2)
weather_stats.columns = ['Total_Crashes', 'Avg_Injuries', 'Avg_Fatalities']
weather_stats = weather_stats.sort_values('Total_Crashes', ascending=False)

weather_fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Crashes by Weather Condition', 'Average Severity by Weather'],
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

# Total crashes by weather
weather_fig.add_trace(
    go.Bar(x=weather_stats.index, y=weather_stats['Total_Crashes'],
           name='Total Crashes'),
    row=1, col=1
)

# Average injuries by weather
weather_fig.add_trace(
    go.Bar(x=weather_stats.index, y=weather_stats['Avg_Injuries'],
           name='Avg Injuries'),
    row=1, col=2
)

weather_fig.update_layout(
    height=500,
    title_text="Weather Condition Analysis",
    title_x=0.5,
    showlegend=False
)


weather_fig.update_xaxes(tickangle=45)


weather_fig.write_html("weather_analysis_visualization.html")
print("Saved: weather_analysis_visualization.html")

Saved: weather_analysis_visualization.html


Insights :

- Majority of the crashes happend during the clear weather followed by cloudy.
- Fog weather has the highest injury rate.

## Driver analysis:

In [ ]:


# Clean age data
df_demo = df_clean.copy()
df_demo['Age_Drv1'] = pd.to_numeric(df_demo['Age_Drv1'], errors='coerce')
df_demo['Age_Drv2'] = pd.to_numeric(df_demo['Age_Drv2'], errors='coerce')

# Create age groups
df_demo['Age_Group_Drv1'] = pd.cut(
    df_demo['Age_Drv1'],
    bins=[0, 25, 35, 50, 65, 100],
    labels=['16-25', '26-35', '36-50', '51-65', '65+']
)

demographics_fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Driver 1 Age Distribution', 'Driver 1 Gender Distribution',
                   'Alcohol Use in Crashes', 'Drug Use in Crashes'],
    specs=[[{"type": "bar"}, {"type": "pie"}],
           [{"type": "pie"}, {"type": "pie"}]]
)

# Age distribution
age_dist = df_demo['Age_Group_Drv1'].value_counts()
demographics_fig.add_trace(
    go.Bar(x=age_dist.index, y=age_dist.values),
    row=1, col=1
)

# Gender distribution
if 'Gender_Drv1' in df_demo.columns:
    gender_dist = df_demo['Gender_Drv1'].value_counts()
    demographics_fig.add_trace(
        go.Pie(labels=gender_dist.index, values=gender_dist.values),
        row=1, col=2
    )

# Alcohol use
if 'AlcoholUse_Drv1' in df_demo.columns:
    alcohol_dist = df_demo['AlcoholUse_Drv1'].value_counts()
    demographics_fig.add_trace(
        go.Pie(labels=alcohol_dist.index, values=alcohol_dist.values),
        row=2, col=1
    )

# Drug use
if 'DrugUse_Drv1' in df_demo.columns:
    drug_dist = df_demo['DrugUse_Drv1'].value_counts()
    demographics_fig.add_trace(
        go.Pie(labels=drug_dist.index, values=drug_dist.values),
        row=2, col=2
    )

demographics_fig.update_layout(
    height=800,
    title_text="Driver Demographics Analysis",
    title_x=0.5,
    showlegend=False
)


demographics_fig.write_html("demographics_visualization.html")
print("Saved demographics_visualization.html")

Saved demographics_visualization.html


Insights:

Age Distribution:

- The 16–25 age group is involved in the most crashes, with over 18,000 incidents—indicating young drivers are a high-risk group.

- A sharp decline in incidents is observed with increasing age; drivers aged 65+ are involved in the fewest crashes.

- The 26–35 and 36–50 age groups also contribute significantly, suggesting mid-age drivers still present notable risk, though less than younger ones.

Gender Distribution

- Male drivers are involved in more crashes (27,093) compared to females , showing a notable gender disparity.

- Unknown gender accounts for a small portion (~4,442), suggesting either missing or non-disclosed data.

Alcohol & Drug Influence

- In crashes, alcohol involvement is evident in ~5% of cases (2,440 out of 51,115).

- Drug use is even lower, observed in less than 1% of cases (447 out of 51,115), showing alcohol is the more prevalent substance-related factor.

## Hotspot Analysis: Density plot, scatterplot.

In [ ]:
hotspots_fig = px.density_mapbox(
    df_clean,
    lat='Latitude',
    lon='Longitude',
    z='Totalfatalities',
    radius=15,
    center=dict(lat=df_clean['Latitude'].mean(), lon=df_clean['Longitude'].mean()),
    zoom=11,
    mapbox_style="open-street-map",
    title="Fatal Crash Hotspots",
    color_continuous_scale="Reds",
    opacity=0.7
)


fatal_crashes = df_clean[df_clean['Totalfatalities'] > 0]
if len(fatal_crashes) > 0:
    hotspots_fig.add_trace(
        go.Scattermapbox(
            lat=fatal_crashes['Latitude'],
            lon=fatal_crashes['Longitude'],
            mode='markers',
            marker=dict(size=8, color='red', opacity=0.6),
            text=fatal_crashes['DateTime'].astype(str) + '<br>' +
                 'Fatalities: ' + fatal_crashes['Totalfatalities'].astype(str),
            hovertemplate='<b>Fatal Crash</b><br>%{text}<extra></extra>',
            name='Fatal Crashes'
        )
    )

hotspots_fig.update_layout(
    height=600,
    title_x=0.5,
    title_font_size=16
)



hotspots_fig.write_html("hotspots_visualization.html")
print("Saved: hotspots_visualization.html")

Saved: hotspots_visualization.html


Insights:

- Fatal crashes are highly concentrated in certain geographic clusters, forming well-defined hotspots.

- These hotspots are likely near urban or high-traffic intersections, suggesting infrastructure or behavior-related risks.

- Some zones see multiple events, implying recurring risk rather than isolated incidents.

In [ ]:

stats = {}

# Basic statistics
stats['total_crashes'] = len(df_clean)
stats['total_injuries'] = df_clean['Totalinjuries'].sum()
stats['total_fatalities'] = df_clean['Totalfatalities'].sum()
stats['avg_injuries_per_crash'] = df_clean['Totalinjuries'].mean()
stats['date_range'] = f"{df_clean['DateTime'].min()} to {df_clean['DateTime'].max()}"

# Geographic coverage
stats['lat_range'] = (df_clean['Latitude'].min(), df_clean['Latitude'].max())
stats['lon_range'] = (df_clean['Longitude'].min(), df_clean['Longitude'].max())

# Most dangerous conditions
stats['worst_weather'] = df_clean.groupby('Weather')['Totalfatalities'].sum().idxmax()
stats['worst_light'] = df_clean.groupby('Lightcondition')['Totalfatalities'].sum().idxmax()
if 'SurfaceCondition' in df_clean.columns:
    stats['worst_surface'] = df_clean.groupby('SurfaceCondition')['Totalfatalities'].sum().idxmax()

# Peak times
stats['peak_hour'] = df_clean['Hour'].mode().iloc[0] if len(df_clean['Hour'].mode()) > 0 else 'N/A'
stats['peak_day'] = df_clean['DayOfWeek'].mode().iloc[0] if len(df_clean['DayOfWeek'].mode()) > 0 else 'N/A'

# Print summary

print(f"Total Crashes Analyzed: {stats['total_crashes']:,}")
print(f"Total Injuries: {stats['total_injuries']:,}")
print(f"Total Fatalities: {stats['total_fatalities']:,}")
print(f"Average Injuries per Crash: {stats['avg_injuries_per_crash']:.2f}")
print(f"Data Period: {stats['date_range']}")
print(f"Geographic Coverage: Lat {stats['lat_range'][0]:.4f} to {stats['lat_range'][1]:.4f}")
print(f"                    Lon {stats['lon_range'][0]:.4f} to {stats['lon_range'][1]:.4f}")
print(f"Most Dangerous Weather: {stats['worst_weather']}")
print(f"Most Dangerous Light Condition: {stats['worst_light']}")
if 'worst_surface' in stats:
    print(f"Most Dangerous Surface: {stats['worst_surface']}")
print(f"Peak Crash Hour: {stats['peak_hour']}:00")
print(f"Peak Crash Day: {stats['peak_day']}")



Total Crashes Analyzed: 51,162
Total Injuries: 23,439.0
Total Fatalities: 161.0
Average Injuries per Crash: 0.46
Data Period: 2012-01-01 07:25:00+00:00 to 2024-03-15 07:11:00+00:00
Geographic Coverage: Lat 33.3199 to 33.4585
                    Lon -111.9791 to -111.8774
Most Dangerous Weather: Clear
Most Dangerous Light Condition: Dark Lighted
Most Dangerous Surface: Dry
Peak Crash Hour: 0:00
Peak Crash Day: Friday
